In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split  , GridSearchCV
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline

In [2]:
train_df = pd.read_csv("../Data/Preprocessed_Train.csv")
test_df = pd.read_csv("../Data/Preprocessed_Test.csv")

print(train_df.columns[train_df.columns != "SalePrice "])

X = train_df[train_df.columns[train_df.columns != "SalePrice"]]
y = train_df[train_df.columns[train_df.columns == "SalePrice"]]

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.3 , shuffle=True , random_state=42)

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       ...
       'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth',
       'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca',
       'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=237)


In [3]:
# Define models to test
models = {
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'Ridge': Ridge(),
    'Lasso': Lasso()
}

In [4]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# Define hyperparameters for tuning
params = {
    'RandomForest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'GradientBoosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0]
    },
    'Ridge': {
        'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
    },
    'Lasso': {
        'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
    }
}

In [6]:
# Initialize a variable to store the best model and its performance
best_model = None
best_score = np.inf

In [7]:
y_train

,SalePrice
135,174000
1452,145000
762,215200
932,320000
435,212000
...,...
1095,176432
1130,135000
1294,115000
860,189950


In [8]:
# Perform hyperparameter tuning and evaluate each model
for model_name in models:
    grid_search = GridSearchCV(models[model_name], params[model_name], cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    # Get the best model from the grid search
    best_estimator = grid_search.best_estimator_

    # Evaluate the model on the test set
    y_pred = best_estimator.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)

    # If this model has the best score so far, save it
    if mse < best_score:
        best_score = mse
        best_model = best_estimator

c:\Users\ashvi\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\ashvi\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [10]:
# Print the best model and its MSE on the test set
print(f"Best Model: {best_model}")
print(f"Root Mean Squared Error: {best_score ** 0.5}")

Best Model: GradientBoostingRegressor(max_depth=7, n_estimators=200, subsample=0.8)
Root Mean Squared Error: 26749.79112223938
